In [3]:
import os
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import PromptAgentDefinition, MCPTool, Tool

load_dotenv()

foundry_project_endpoint = os.getenv("FOUNDRY_PROJECT_ENDPOINT")
model_deployment_name = os.getenv("MODEL_DEPLOYMENT_NAME")
mcp_server_name = os.getenv("MCP_SERVER_NAME")

In [4]:
project_client = AIProjectClient(
    endpoint=foundry_project_endpoint,
    credential=DefaultAzureCredential(),
)

In [5]:
openai_client = project_client.get_openai_client()

In [6]:
connection_id = ""

for connection in project_client.connections.list():
    if connection.name == mcp_server_name:
        connection_id = connection.id
        break

print(f"The MCP Server Connection ID is: {connection_id}")

The MCP Server Connection ID is: /subscriptions/3c1227d7-cdd2-4efc-985f-e304cc9273ea/resourceGroups/foundarydemorg/providers/Microsoft.CognitiveServices/accounts/demoprojectfractal/projects/demo-proj/connections/mslearn-mcp-server


In [7]:
tool = MCPTool(
    server_label = "microsoft_learn_server",
    server_url="https://learn.microsoft.com/api/mcp",
    require_approval="never",
    project_connection_id=connection_id
)

In [8]:
agent = project_client.agents.create_version(
    agent_name="MCP-Agent",
    definition=PromptAgentDefinition(
        model=model_deployment_name,
        instructions="You are an intelligent assistant that can interact with the Microsoft Learn MCP server to provide users with relevant learning resources and information about Microsoft technologies.",
        tools=[tool],
    )
)

print(f"Created MCP Agent with ID: {agent.id}")

Created MCP Agent with ID: MCP-Agent:1


In [9]:
# create a conversation to use with the agent
conversation = openai_client.conversations.create()
print(f"Created conversation with id: {conversation.id}")

Created conversation with id: conv_c2051d71a40f391700x76nph1xqqquuL3IXckv8BzXDGXv3pRd


In [ ]:
user_query = "Find me learning paths on Azure AI services for building intelligent applications." 

In [11]:
response = openai_client.responses.create(
    conversation=conversation.id,
    extra_body={
        "agent": {
            "name": "MCP-Agent",
            "type": "agent_reference"
        }
    },
    input=user_query
)

print(f"Agent Response: {response.output_text}")

Agent Response: Here are some learning paths on Azure AI services for building intelligent applications:

1. **[Get started with Azure AI Services](https://learn.microsoft.com/en-us/training/paths/get-started-azure-ai/)**
   - Explore how to provision, secure, monitor, and deploy Azure AI services resources, and use them to build intelligent solutions.

2. **[Develop natural language solutions in Azure](https://learn.microsoft.com/en-us/training/paths/develop-language-solutions-azure-ai/)**
   - Learn to interpret the semantic meaning of written or spoken language and create speech-enabled apps with Azure AI services.

3. **[Develop AI agents on Azure](https://learn.microsoft.com/en-us/training/paths/develop-ai-agents-on-azure/)**
   - Understand AI agents, including when to use them and how to build them using Azure AI Foundry Agent Service and Semantic Kernel Agent Framework.

4. **[Build AI Apps with Azure Database for PostgreSQL](https://learn.microsoft.com/en-us/training/paths/bui